In [1]:
import argparse

from sentence_transformers.losses import CosineSimilarityLoss
from datasets import load_dataset
from functools import partial
import evaluate

from ZeroBERTo.modeling_zeroberto import ZeroBERToModel, ZeroBERToDataSelector
from ZeroBERTo.trainer import ZeroBERToTrainer

import json
from datetime import datetime

def compute_metrics(y_pred, y_test, metrics):
    results = {}
    try:
        for average in ['weighted', 'macro']:
            results[average] = {}
            for metric_name in metrics.keys():
                if metric_name == "accuracy":
                    results[average].update(metrics[metric_name].compute(predictions=y_pred, references=y_test))
                else:
                    results[average].update(metrics[metric_name].compute(predictions=y_pred, references=y_test, average=average))
    except:
        print("Error")
    return results

def arg_parse() -> argparse.Namespace:
    parser = argparse.ArgumentParser(
        description="Training and evaluation",
        formatter_class=argparse.ArgumentDefaultsHelpFormatter,
    )
    parser.add_argument(
        "--model_name_or_path", type=str, help="Model name", default="sentence-transformers/paraphrase-mpnet-base-v2"
    )
    parser.add_argument(
        "--dataset", type=str, help="Dataset name", default="SetFit/sst2"
    )
    parser.add_argument(
        "--dataset_train_split", type=str, help="Training data split", default="train"
    )
    parser.add_argument(
        "--dataset_test_split", type=str, help="Test data split", default="test"
    )
    parser.add_argument(
        "--hypothesis_template", type=str, help="Hypothesis Template for First Shot classification", default="{}"
    )
    parser.add_argument(
        "--multi_target_strategy", type=str, help="Multi Target Strategy", default=None
    )
    parser.add_argument(
        "--use_differentiable_head", action=argparse.BooleanOptionalAction, help="Use Differentiable head", default=False
    )
    parser.add_argument(
        "--num_iterations", type=int, help="Number of pairs to generate on training.", default=20
    )
    parser.add_argument(
        "--num_setfit_iterations", type=int, help="Number of SetFit training iterations to perform", default=2
    )   
    parser.add_argument(
        "--num_epochs", type=int, help="Number of self-training loop iterations to perform", default=1
    )  
    parser.add_argument(
        "--samples_per_label", type=int, help="Number of samples per class to pick for training", default=4
    )   
    parser.add_argument(
        "--normalize_embeddings", type=bool, help="Normalize Embeddings", default=False
    )
    parser.add_argument(
        "--selection_strategy", type=str, help="How Data Selector should pick training data", default='top_n'
    )
    parser.add_argument(
        "--batch_size", type=int, help="Batch size for GPU", default=8
    )
    parser.add_argument(
        "--var_samples_per_label", type=int, nargs="*", default=None
    )
    parser.add_argument(
        "--var_selection_strategy", type=str, nargs="*", default=None
    )
    parser.add_argument(
        "--learning_rate", type=float,  default=2e-5
    )
    parser.add_argument(
        "--body_learning_rate", type=float, default=1e-6
    )
    parser.add_argument(
        "--num_body_epochs", type=int, default=2
    )
    parser.add_argument(
        "--freeze_head",help="If True, will train head.", default=False,action=argparse.BooleanOptionalAction
    )
    parser.add_argument(
        "--freeze_body",help="If True, will not train body.", default=False,action=argparse.BooleanOptionalAction
    )
    parser.add_argument(
        "--train_first_shot",help="If True, will train once before First Shot.", default=False,action=argparse.BooleanOptionalAction
    )    
    parser.add_argument(
        "--allow_resampling",help="If True, will not discard training data on subsequent iterations.", default=False,action=argparse.BooleanOptionalAction
    )              
    parser.add_argument(
        "--random_seed",help="Integer as random seed for dataset sampling.", default=42,type=int
    )
    parser.add_argument(
        "--train_dataset_size",help="number of unlabeled samples to take as train dataset", default=5000,type=int
    ) 
    parser.add_argument(
        "--auto",help="If True, will automatically set hyperparameters.", default=False,action=argparse.BooleanOptionalAction
    ) 

    args = parser.parse_args()
    return args

/Users/alealcoforado/opt/anaconda3/envs/venv_zeroberto/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'ZeroBERTo'

In [14]:
import pandas as pd
dfs = pd.DataFrame()
for seed in [10,20,30,40,50,60,70,80,90,100]:
    try:
        df = pd.read_csv(r'/Users/alealcoforado/Downloads/iara-project-news-articles-ptbr-dataset_8_xx.csv'.replace("xx",str(seed)))
        dfs = pd.concat([dfs,df],axis=0)
        print(seed)
    except:
        pass
dfs

10
20
30
40
50
60
70
80


,Unnamed: 0,eval_loss,eval_accuracy,eval_precision_weighted,eval_recall_weighted,eval_f1_weighted,eval_precision_macro,eval_recall_macro,eval_f1_macro,eval_runtime,eval_samples_per_second,eval_steps_per_second,epoch
0,0,2.033733,0.672000,0.740763,0.672000,0.686733,0.543586,0.718780,0.573872,1662.9175,18.041,4.510,30.0
0,0,2.146754,0.680367,0.771346,0.680367,0.701150,0.543132,0.743573,0.575151,1660.5832,18.066,4.516,30.0
0,0,2.179196,0.675000,0.756965,0.675000,0.697206,0.567310,0.747164,0.601254,1661.3978,18.057,4.514,30.0
0,0,2.462106,0.621800,0.742720,0.621800,0.652686,0.532274,0.716920,0.561760,1662.4475,18.046,4.511,30.0
0,0,2.412998,0.649867,0.725933,0.649867,0.663165,0.522772,0.740875,0.565486,1664.2546,18.026,4.507,30.0
0,0,2.245385,0.679733,0.772105,0.679733,0.693870,0.549283,0.736441,0.582601,1664.6270,18.022,4.506,30.0
0,0,2.550590,0.629400,0.728714,0.629400,0.651504,0.490073,0.712330,0.521117,1663.3841,18.036,4.509,30.0
0,0,2.252890,0.661867,0.755516,0.661867,0.683938,0.521435,0.723108,0.555962,1664.4527,18.024,4.506,30.0


In [16]:
dfs['eval_f1_macro'].mean(),dfs['eval_f1_macro'].std()

(0.5671504597131501, 0.023259410810150448)